In [1]:
# Useful starting lines
%matplotlib inline
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
import sys
import matplotlib.pyplot as plt
from implementations import *

# TO REMOVE ONCE IMPLEMENTED
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

%load_ext autoreload
%autoreload 2

# Data input and output paths
DATA_TRAIN_PATH = '../data/train.csv' 
DATA_TEST_PATH = '../data/test.csv'
OUTPUT_PATH = 'predictions_out.csv'

# For debugging purpose
np.set_printoptions(threshold= 50)
np.random.seed(10)

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

COLUMNS_NAMES = ['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt']

## Analyse Data


### Null values

In [ ]:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
data = pd.read_csv(DATA_TRAIN_PATH, header=0, na_values='-999', delimiter=',')

indicesS = data[data['Prediction'].str.contains('s')].index
indicesB = pd.Int64Index(np.arange(len(data))).difference(indicesS)
data.loc[indicesS, 'Prediction'] = np.ones(len(indicesS))
data.loc[indicesB, 'Prediction'] = np.ones(len(indicesB))-2

sns.heatmap(data.isna())
display(data.head())
display(data.describe())


### Plot Correlation Matrix

In [ ]:
corr = data.drop(['Id'], axis=1).corr()
fig, (ax) = plt.subplots(1, 1, figsize=(15,10))

hm = sns.heatmap(corr, 
                 ax=ax,           # Axes in which to draw the plot, otherwise use the currently-active Axes.
                 cmap="coolwarm", # Color Map.
                 #square=True,    # If True, set the Axes aspect to “equal” so each cell will be square-shaped.
                 annot=True, 
                 fmt='.2f',       # String formatting code to use when adding annotations.
                 #annot_kws={"size": 14},
                 linewidths=.05)

fig.subplots_adjust(top=0.93)
fig.suptitle('Wine Attributes Correlation Heatmap', 
              fontsize=14, 
              fontweight='bold')


### Outliers 

In [ ]:
data[~data.isna()].drop(['Id'], axis = 1).boxplot(figsize=(20, 10), whis=2.5)

In [ ]:
data[data['PRI_jet_num']==3].describe()

## Clean the data

In [3]:
def outliers_iqr(tX, whos = 2.5):   
    for i, column in enumerate(tX.T):
        # Compute lower and upper bounds depending on the 1st and 3st quartile
        quartile_1, quartile_3 = np.nanpercentile(column, [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr * whos)
        upper_bound = quartile_3 + (iqr * whos)
        
        # Increase small outliers
        indices = np.where(column < lower_bound)
        tX[indices, i] = lower_bound
        
        # Decrease big outliers
        indices = np.where(column > upper_bound)
        tX[indices, i] = upper_bound
        
    return tX

def infereMissingValues(tX, col):
    #print("col: ", col)
    # Regression parameters
    
    max_iters = 500
    gamma = 0.001
    lambda_ = 0.01
    degree = 3
    
    # Get all rows where DER_Mass_MMC is undefined and respectively defined
    train = tX[~np.isnan(tX[:, col]), :]
    test = tX[np.isnan(tX[:, col]), :]  
    
    # Create the training and testing sets
    tx_0_train = np.delete(train, col, axis=1)
    tx_0_train = tx_0_train[:, ~np.isnan(np.delete(tX, col, axis=1)).any(axis=0)]
    ty_0_train = train[:, col]
    tx_0_test =  np.delete(test, col, axis=1)
    tx_0_test = tx_0_test[:, ~np.isnan(np.delete(tX, col, axis=1)).any(axis=0)]
    
    # Expande the dimension with polynomial
    tx_0_train = build_poly(tx_0_train, degree)
    tx_0_test =build_poly(tx_0_test, degree)
    
    initial_w = np.zeros(tx_0_train.shape[1])
    # Train the model
    loss, weights = ridge_regression(ty_0_train, tx_0_train, lambda_)
    #loss, weights = least_squares_GD(ty_0_train, tx_0_train, initial_w, max_iters, gamma)
    #loss, weights = least_squares(ty_0_train, tx_0_train)
    # Predict the undefined values
    tx_0_test = np.dot(tx_0_test, weights)
    tX[np.isnan(tX[:, col]), col]  = tx_0_test
    print("Loss: ", loss)  
    return tX

def pri_jet_split(y, tX, ids):
    # Get the rows' indexes of each category
    indices_cat_0 = np.argwhere(tX[:, 22] == 0).flatten()
    indices_cat_1 = np.argwhere(tX[:, 22] == 1).flatten()
    indices_cat_2 = np.argwhere(tX[:, 22] == 2).flatten()
    indices_cat_3 = np.argwhere(tX[:, 22] == 3).flatten()

    # Split the dataset in 4
    tXC = [tX[indices_cat_0, :], tX[indices_cat_1, :], tX[indices_cat_2, :], tX[indices_cat_3, :]]
    idsC = [ids[indices_cat_0], ids[indices_cat_1], ids[indices_cat_2], ids[indices_cat_3]]
    yC = [y[indices_cat_0], y[indices_cat_1], y[indices_cat_2], y[indices_cat_3]]
    
    # Delete undefined features for each category, cf. features description : http://opendata.cern.ch/record/328
    # And the PRI_jet_num feature which is constant now
    tXC[0] = np.delete(tXC[0], (4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29), 1)
    tXC[1] = np.delete(tXC[1], (4, 5, 6, 12, 22, 26, 27, 28), 1)
    tXC[2] = np.delete(tXC[2], (22), 1)
    tXC[3] = np.delete(tXC[3], (22), 1)
    
    return yC, tXC, idsC

def prepareData(y, tX, ids):
    # Replace -999 by nan
    tX[tX==-999]=np.nan
    
    # Cap our outliers records.
    tX = outliers_iqr(tX)
      
    # Split the data into 4 differents class, depending on PRI_jet_num value
    yC, tXC, idsC = pri_jet_split(y, tX, ids)
    
    # Drop uncorreled features
    #tX = np.delete(tX, (0, 2, 7, 8, 14, 15, 16, 17, 18, 19, 20, 24, 25, 27, 28), 1)
    #np.corrcoef(x, y)
    
    
    for i in range(len(tXC)):
        # Standardize the data
        tXC[i], _, _ = standardize(tXC[i])
        # Infere missing values on each column (normally just DER_Mass_MMC)     
        colsWithNan = np.unique(np.where(np.isnan(tXC[i]))[1])
        for col in colsWithNan:
            tXC[i] = infereMissingValues(tXC[i], col)
        
        # Build polynomial extension
        tXC[i] = build_poly(tXC[i], 13)
        
    return yC, tXC, idsC

clean_yC, clean_tX, clean_ids = prepareData(y, tX, ids)

Loss:  0.029042294453335133
Loss:  0.03142717484550267
Loss:  0.036041493854561624
Loss:  0.03225699119398734


## Split the data into, training, validation and test sets

In [4]:
def splitTestTrainValid(y, clean_tX, ids):
    # How much annotated data for training and validation. The rest is used for testing.
    training_perc, validation_perc = 0.4, 0.1

    indices = np.arange(len(y))
    np.random.shuffle(indices)

    splits = (np.array([training_perc, validation_perc]) * len(y)).astype(int).cumsum()
    training_indices, validation_indices, test_indices = np.split(indices, splits)

    tX_train = clean_tX[training_indices]
    y_train = y[training_indices]
    ids_train = ids[training_indices]

    tX_validation = clean_tX[validation_indices]
    y_validation = y[validation_indices]
    ids_validation = ids[validation_indices]

    tX_test = clean_tX[test_indices]
    y_test = y[test_indices]
    ids_test = ids[test_indices]
    
    return tX_train, y_train, tX_validation, y_validation, tX_test, y_test

## Parameter Exploration

In [17]:
# Split the data in test and train sets
txC_train = []
yC_train = []
txC_test = []
yC_test = []

for i in range(len(clean_tX)):
    tX_train, y_train, tX_validation, y_validation, tX_test, y_test = splitTestTrainValid(clean_yC[i], clean_tX[i], clean_ids[i])
    txC_train.append(tX_train)
    yC_train.append(y_train)
    txC_test.append(tX_test)
    yC_test.append(y_test)

# Models parameters

#initial_w = np.random.normal(0, 0.1, [tX.shape[1],])
max_iters = 1000
gamma = 0.0000000001
lambda_ = 0.01
degree = 3

initial_w = np.zeros(txC_train[0].shape[1])
loss_0, weights_0 = least_squares_GD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
initial_w = np.zeros(txC_train[1].shape[1])
loss_1, weights_1 = least_squares_GD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
initial_w = np.zeros(txC_train[2].shape[1])
loss_2, weights_2 = least_squares_GD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
initial_w = np.zeros(txC_train[3].shape[1])
loss_3, weights_3 = least_squares_GD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
#ridge_regression(yC_train[0], txC_train[0], lambda_)
#least_squares_GD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
#least_squares_SGD(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
#least_squares(yC_train[0], txC_train[0])
#logistic_regression(yC_train[0], txC_train[0], initial_w, max_iters, gamma)
#reg_logistic_regression(yC_train[0], txC_train[0], lambda_, initial_w, max_iters, gamma)

predictions_0 = predict_labels(weights_0, txC_test[0])
predictions_1 = predict_labels(weights_1, txC_test[1])
predictions_2 = predict_labels(weights_2, txC_test[2])
predictions_3 = predict_labels(weights_3, txC_test[3])

print("", classification_report(yC_test[0],predictions_0))
print("", classification_report(yC_test[1],predictions_1))
print("", classification_report(yC_test[2],predictions_2))
print("", classification_report(yC_test[3],predictions_3))

w:  [0. 0. 0. ... 0. 0. 0.]
err:  [ 1.  1. -1. ... -1.  1. -1.]
w:  [-4.86750907e-11  1.90357751e-11 -3.70290634e-11 ...  2.73845003e-04
 -3.90890308e-10 -1.31717115e-02]
err:  [ 1.00283242  0.99817292 -2.09016755 ... -0.97806302 19.96208341
 -0.99908883]
w:  [3.84135510e+03 1.25540585e+04 3.18113398e+03 ... 6.11549525e+11
 3.67791561e+05 5.38657068e+15]
err:  [ 1.13926138e+13  5.52573519e+09  1.29738831e+18 ... -2.59285839e+14
 -8.07546831e+18 -3.71424524e+12]
w:  [-4.64958689e+21 -1.51954833e+22 -3.85048165e+21 ... -7.40200383e+29
 -4.45177854e+23 -6.51996631e+33]
err:  [-1.37897256e+31 -6.68815636e+27 -1.57036961e+36 ...  3.13841691e+32
  9.77463843e+36  4.49574227e+30]
w:  [5.62789860e+39 1.83927391e+40 4.66065497e+39 ... 8.95944692e+47
 5.38846972e+41 7.89182138e+51]
err:  [ 1.66911984e+49  8.09540001e+45  1.90078842e+54 ... -3.79876590e+50
 -1.18313035e+55 -5.44168378e+48]
w:  [-6.81205521e+57 -2.22627242e+58 -5.64129549e+57 ... -1.08445890e+66
 -6.52224851e+59 -9.55232615e+69]
e

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan .

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan .

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ... nan nan nan]
err:  [nan nan nan ... nan nan nan]
w:  [nan nan nan ...

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 287 is different from 235)

## Regression 

In [ ]:
#initial_w = np.zeros(clean_tX.shape[1])
max_iters = 3000
gamma = 0.01
lambda_ = 0.01

loss_0, weights_0 = ridge_regression(clean_yC[0], clean_tX[0], lambda_)
loss_1, weights_1 = ridge_regression(clean_yC[1], clean_tX[1], lambda_)
loss_2, weights_2 = ridge_regression(clean_yC[2], clean_tX[2], lambda_)
loss_3, weights_3 = ridge_regression(clean_yC[3], clean_tX[3], lambda_)

#loss, weights = least_squares(y, clean_tX)
#loss, weights = ridge_regression(y, clean_tX, lambda_)
predictions_0 = predict_labels(weights_0, clean_tX[0])
predictions_1 = predict_labels(weights_1, clean_tX[1])
predictions_2 = predict_labels(weights_2, clean_tX[2])
predictions_3 = predict_labels(weights_3, clean_tX[3])

print("", classification_report(clean_yC[0],predictions_0))
print("", classification_report(clean_yC[1],predictions_1))
print("", classification_report(clean_yC[2],predictions_2))
print("", classification_report(clean_yC[3],predictions_3))

## Generate predictions and save ouput in csv format for submission:

In [ ]:
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


In [ ]:
test_yC, test_tX, test_ids = prepareData(y_test, tX_test, ids_test)

In [ ]:
y_pred_0 = predict_labels(weights_0, test_tX[0])
y_pred_1 = predict_labels(weights_1, test_tX[1])
y_pred_2 = predict_labels(weights_2, test_tX[2])
y_pred_3 = predict_labels(weights_3, test_tX[3])
y_pred = np.concatenate([y_pred_0, y_pred_1, y_pred_2, y_pred_3])
test_ids = np.concatenate([test_ids[0], test_ids[1], test_ids[2], test_ids[3]])

In [ ]:
create_csv_submission(test_ids, y_pred, OUTPUT_PATH)
